In [13]:
# imports
import seaborn as sns
import pandas as pd
import numpy as np
import pickle
import librosa
import librosa.display
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from glob import glob
import warnings
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import os

warnings.filterwarnings("ignore")

n_fft = 1024
hop_length = 315
data_dir = "../data"

In [40]:
def get_files(dir: str):
    """
    Return audio files from a given directory.

    Keyword arguments:
    dir -- the directory containing the audio files
    """
    labs = glob(dir + "/*.lab")
    mp3s = glob(dir + "/*.mp3")
    wavs = glob(dir + "/*.wav")
    return labs, mp3s, wavs


def parse(data, mp3s, wav, dir):
    sound = SoundBites(
        f"{dir}/{data[0]}",
        extension="mp3" if f"{dir}/{data[0]}.mp3" in mp3s else "wav",
    )
    # removes title and sentinel lines
    for line in data[1:-2]:
        sound.timestamps.append(line.split("\t"))
    return sound


def match_labels_with_files(path: str):
    labs, mp3s, wavs = get_files(path)
    data = []
    for l in labs:
        with open(l) as file:
            input_string = file.read()
        data.append(parse([x for x in input_string.split("\n")], mp3s, wavs, path))
    return data


def get_annotations_for_block(
    annotations: list, block_start: float, block_finish: float, threshold=0
):

    assert (
        block_start < block_finish
    ), f"block_start ({block_start}) needs to be less than block_finish ({block_finish})"

    timeframes = []
    # annotation_filename = filename[:-4] +'.lab'
    # filename: whatever.mp3 or wav -> filename[0:-4] + .lab

    overlap = 0

    # for frame in open(annotation_filename, 'r').readlines()[1:-1]:  # parse everything except title and end .
    for frame in annotations:
        split = [part.strip() for part in frame.split("\t")]

        assert len(split) == 3, "expected 3 fields in each line, got: " + str(
            len(split)
        )

        frame_start = float(split[0])
        frame_end = float(split[1])
        label = split[2]

        if block_start > frame_end:
            continue
        elif block_finish < frame_start:
            break
        else:
            # start calculations

            if (
                frame_end - frame_start >= threshold and label == "nobee"
            ):  # ignores too short thresholds

                if (
                    frame_start > block_start
                    and frame_start <= block_finish
                    and frame_end >= block_finish
                ):
                    # frame overlaps with right of block
                    # |____________########|########
                    # bs          tp0      bf      tp1
                    overlap += block_finish - frame_start

                elif (
                    frame_start <= block_start
                    and frame_end > block_start
                    and frame_end < block_finish
                ):
                    # frame overlaps with left of block
                    # #####|########_____|
                    # tp0  bs     tp1    bf
                    overlap += frame_end - block_start

                elif (
                    frame_start > block_start
                    and frame_start <= block_finish
                    and frame_end > block_start
                    and frame_end <= block_finish
                ):
                    # frame completely inside block
                    # |_____########_____|
                    # bs   tp0    tp1    bf
                    overlap += frame_end - frame_start

                elif frame_start <= block_start and frame_end > block_finish:
                    # frame completely surrounds block
                    #  ####|############|####
                    # tp0  bs           bf  tp1
                    overlap += block_finish - block_start

    final_label = "nobee" if overlap > threshold else "bee"
    # return timeframes
    return [overlap / (block_finish - block_start), final_label]


def get_blocks_from_filename(filename: str, block_size: int, threshold: float):
    """
    Returns array of all blocks in provided audiofile
    Each returned block in the array is length of 2:
    - [0] is sound data
    - [1] is bee/nobee annotation
    """
    print(block_size, filename)
    annotation_filename = filename[:-4] + ".lab"
    annotations = open(annotation_filename, "r").readlines()[1:-1]
    blocks = []
    offset = 0
    temp = filename.split("/")[-1]
    print(f"Reading audio file {temp}")
    while True:
        # clear_output()
        print(f"\Attempting to read from {offset}s to {offset+block_size}s...")
        try:
            clip, sr = librosa.core.load(filename, offset=offset, duration=block_size)
        except ValueError as error:
            clip = np.arange(0)

        # print(clip.shape[0] / sr)
        if clip.shape[0] >= block_size * sr:
            annotation = get_annotations_for_block(
                annotations, offset, offset + block_size, threshold
            )
            newBlock = block(annotation[0], annotation[1], clip)
            blocks.append(newBlock)
            # ret.append([clip, annotation])
            block_name = f"{temp[:-4]} timestamp{offset}-{offset+block_size} {annotation[1]} {round(annotation[0], 5)}"
            print(block_name)

        elif clip.shape[0] > 0:
            annotation = get_annotations_for_block(
                annotations, offset, offset + block_size, threshold
            )
            # ret.append([np.pad(clip, (0, block_size*sr-clip.shape[0]), mode='reflect'), annotation])
            newBlock = block(
                annotation[0],
                annotation[1],
                np.pad(clip, (0, block_size * sr - clip.shape[0]), mode="reflect"),
            )
            blocks.append(newBlock)
            block_name = f"{temp[:-4]} timestamp {offset}-{round(clip.shape[0]/sr, 2)} {annotation[1]} {round(annotation[0], 5)}"
            print(block_name)
            print(f"----- Finished {temp}")
            break

        else:
            print(f"----- Finished {temp}")
            break

        offset += block_size
    return blocks

In [41]:
class block:
    def __init__(self, weight, nobee, clip=[]):
        self.nobee = nobee
        self.weight = weight
        self.clip = clip


class SoundBites:
    timestamps = []
    file_path = ""
    extension = ""

    def __init__(self, file_path, extension="mp3", timestamps=[]):
        self.file_path = file_path
        self.extension = extension
        self.timestamps = []

    def __str__(self):
        tmpstr = f"Directory: {self.file_path}\nSound Type: {self.extension}\n{self.timestamps}"
        return tmpstr